In [11]:
from database_settings import spark_utilities
from pyspark.sql.functions import col, lpad
import pandas as pd
import time
from database_settings import mongo_utilities

In [2]:
# Get all headings in the data
df = spark_utilities.get_spark_df('peru_exports')
hs_data = df.select('PART_NANDI')\
    .distinct()\
    .withColumn("HS", lpad(col("PART_NANDI").cast("string"), 10, "0")) \
    .select('HS')\
    .toPandas()
hs_data

,HS
0,2615100000
1,2701110000
2,2613900000
3,0708200000
4,2515110000
...,...
6466,8523803000
6467,1602391000
6468,8418699200
6469,8463101000


In [3]:
# Headings to work with:
# 07XXXXXXXX: Hortalizas, plantas, raíces y tubérculos alimenticios
# 08XXXXXXXX: Frutas y frutos comestibles; cortezas de agrios (cítricos), melones o sandías.

hs_list = ['07', '08']
hs_to_work = hs_data[hs_data['HS'].str.startswith(tuple(hs_list))].sort_values(by='HS', ascending= True)
hs_to_work

,HS
1631,0701100000
5199,0701900000
1633,0702000000
3200,0703100000
123,0703201000
...,...
6181,0813300000
3527,0813400000
5833,0813500000
179,0814001000


In [4]:
# Get heading labels from the headings table obtained online
with open('../../data/temporal_landing/support/NANDINA.TXT', 'r') as f:
    file_lines = f.readlines()
    file_lines = [string.rstrip('\t\n') for string in file_lines][1:]
    file_lines = [string.split('\t') for string in file_lines]
    file_lines = [[element for element in inner_list if element.strip()] for inner_list in file_lines]

hs_labels =  pd.DataFrame(file_lines,
                          columns=['PARTIDA','DESCRIP','ADVAL','IGV','ISC','SEGURO','CUODE','CIIU','FINICIO', 'FFINAL']).rename(columns={'PARTIDA':'HS'})[['HS','DESCRIP']]
hs_labels['HS'] = hs_labels['HS'].astype(str).str.zfill(10)
hs_labels

,HS,DESCRIP
0,3926909030,ESCAFANDRAS Y MASCARAS DE PROTECCION
1,3926909090,DEMAS MANUFACTURAS DE PLASTICO
2,4001210000,HOJAS AHUMADAS
3,4001220000,CAUCHOS TECNICAMENTE ESPECIFICADOS (TSNR)
4,4001291000,HOJAS DE CREPE
...,...,...
24952,3002131000,- - - Para tratamiento oncológico o VIH
24953,3002141000,- - - Para tratamiento oncológico o VIH
24954,9811000000,- MATERIAL DE GUERRA ...
24955,0208900010,"- - CARNE DE CUY (COBAYO, CONEJILLO DE INDIAS)..."


In [5]:
# Join both dataframes in the headings column
my_hs = pd.merge(hs_to_work,hs_labels, on= 'HS',how='left').sort_values(by='HS', ascending=True)
my_hs

,HS,DESCRIP
0,0701100000,PAPAS(PATATAS) FRESCAS O REFRIGERADOS PARA SIE...
1,0701100000,- Para siembra
2,0701100000,- Para siembra
3,0701900000,PAPAS(PATATAS) FRESCAS O REFRIGERADAS (EXC. PA...
4,0701900000,- Las demas
...,...,...
367,0813500000,"- Mezclas de frutas u otros frutos, secos,..."
366,0813500000,MEZCLA DE FRUTOS SECOS O FRUTOS D'CASCARA DEL ...
368,0813500000,"- Mezclas de frutas u otros frutos, secos, o d..."
369,0814001000,"- De limón (limón sutil, limón común, limón cr..."


In [6]:
# Verify that no headings code has a null description
my_hs[my_hs.isnull().any(axis=1)]

,HS,DESCRIP


In [7]:
# See how many descriptions are available per headings code in the data of interest
my_hs['HS'].value_counts()

0711900000    4
0805209000    4
0712909000    4
0802900000    4
0703900000    4
             ..
0805299000    1
0805291000    1
0805220000    1
0805210000    1
0814009000    1
Name: HS, Length: 174, dtype: int64

In [8]:
# Eliminate multiple spacings in the description
my_hs['DESCRIP'].replace(r'\s+', ' ', regex=True, inplace=True)
# Eliminate the dashes at the beginning of the description
my_hs['DESCRIP'].replace(r'^[-\s]+', '', regex=True, inplace=True)
# Add final dot to description
my_hs['DESCRIP'] = my_hs['DESCRIP'].apply(lambda x: x if x.endswith('.') else x+'.')
# Group all descriptions from a heading in a single cell
my_hs = my_hs.groupby('HS')['DESCRIP'].agg(lambda x: ' '.join(x)).reset_index()
# Rename the description
my_hs.rename(columns={'DESCRIP':'MERGED_DESCRIP'}, inplace=True)
# Add additional columns for curated description and mappings
my_hs['CURATED_DESCRIP'] = ''
my_hs['MAPPED_TO'] = '' # It may be the case that some headings in the list did not exist in the PDF, so the must be mapped to another headings that do exist.

In [11]:
# Export to CSV for manual resolution of descriptions
my_hs.to_csv('./hs_curated_'+str(int(time.time()*1000))+'.csv', index=None)

In [22]:
# Import the manually curated CSV
curated_hs = pd.read_csv('./hs_curated_1680276488570.csv')

In [23]:
curated_hs

,HS,MERGED_DESCRIP,CURATED_DESCRIP,MAPPED_TO
0,701100000.0,PAPAS(PATATAS) FRESCAS O REFRIGERADOS PARA SIE...,"Papas (patatas) para siembra, frescos o refrig...",NaN
1,701900000.0,PAPAS(PATATAS) FRESCAS O REFRIGERADAS (EXC. PA...,"Papas (patatas) excepto para siembra, frescos ...",NaN
2,702000000.0,TOMATES FRESCOS O REFRIGERADOS. Tomates fresco...,Tomates frescos o refrigerados,NaN
3,703100000.0,Cebollas y chalotes. CEBOLLAS Y CHALOTES. Cebo...,Cebollas y chalotes frescos o refrigerados,NaN
4,703201000.0,Para siembra.,Ajos para siembra frescos o refrigerados,NaN
...,...,...,...,...
170,813400000.0,Las demas frutas u otros frutos. Las demás fru...,Otros frutas o frutos secos,NaN
171,813500000.0,"Mezclas de frutas u otros frutos, secos, o de ...",Mezcla de frutas o frutos secos,NaN
172,814001000.0,"De limón (limón sutil, limón común, limón crio...","Corteza de (limón sutil, limón común, limón cr...",NaN
173,814009000.0,Las demás.,Otras cortezas de agrios (cítricos),NaN


In [24]:
# Add it to the database
db = mongo_utilities.connect() # Connect to the database

Database connected successfully!


In [25]:
collection = db['peru_exports_headings'] # Use (or create) the collection]

In [26]:
# Convert the dataframe into a dict
curated_hs = curated_hs.to_dict('records')
# Insert the data
try:
    result = collection.insert_many(curated_hs)
    print("Headings added to database")
except Exception as e:
    print("An exception occurred ::", e)

Headings added to database
